In [29]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, IntSlider

# Define the continuous signal (original)
fs_continuous = 1000  # High sampling rate for "continuous" signal
t_continuous = np.linspace(0, 1, fs_continuous, endpoint=False)  # 1 second
f_signal = 20  # Signal frequency in Hz
signal_continuous = np.sin(2 * np.pi * f_signal * t_continuous)

# Define different sampling rates
fs_high = 50  # Proper sampling (> 2*f_signal)
fs_nyquist = 40  # Nyquist rate (2*f_signal)
fs_low = 25  # Undersampling (aliasing occurs)

# Sample the signal at different rates
t_high = np.arange(0, 1, 1/fs_high)
signal_high = np.sin(2 * np.pi * f_signal * t_high)

t_nyquist = np.arange(0, 1, 1/fs_nyquist)
signal_nyquist = np.sin(2 * np.pi * f_signal * t_nyquist)

t_low = np.arange(0, 1, 1/fs_low)
signal_low = np.sin(2 * np.pi * f_signal * t_low)

# Function to perform sinc interpolation
def sinc_interp(x, s, t):
    T = s[1] - s[0]  # Sampling interval
    sinc_matrix = np.tile(x, (len(s), 1)) - np.tile(s[:, np.newaxis], (1, len(x)))
    return np.dot(np.transpose(np.sinc(sinc_matrix / T)), t)

# Function to compute power spectrum in dB
def compute_power_spectrum(signal, fs):
    freqs = np.fft.fftfreq(len(signal), 1/fs)
    power_spectrum = 10 * np.log10(np.abs(np.fft.fft(signal))**2)
    mask = (freqs >= 0) & (freqs <= 80)
    return freqs[mask], power_spectrum[mask]

# Function to update the plot based on the resampling frequency
def update_plot(fs_resample):
    # Interpolate the sampled signals
    signal_high_interp = sinc_interp(t_continuous, t_high, signal_high)
    signal_nyquist_interp = sinc_interp(t_continuous, t_nyquist, signal_nyquist)
    signal_low_interp = sinc_interp(t_continuous, t_low, signal_low)

    # Resample the signal
    t_resample = np.arange(0, 1, 1/fs_resample)
    signal_resample = np.sin(2 * np.pi * f_signal * t_resample)
    signal_resample_interp = sinc_interp(t_continuous, t_resample, signal_resample)

    # Plot the original and sampled signals
    plt.figure(figsize=(12, 6))

    # Original continuous signal
    plt.plot(t_continuous, signal_continuous, 'k', alpha=0.5, label="Original Signal (1000 Hz)")

    # Properly sampled signal
    plt.plot(t_high, signal_high, 'bo', label=f"Properly Sampled ({fs_high} Hz)", zorder=3)

    # Nyquist rate sampling
    plt.plot(t_nyquist, signal_nyquist, 'go', label=f"Nyquist Sampling ({fs_nyquist} Hz)", zorder=3)

    # Undersampled signal (Aliasing)
    plt.plot(t_low, signal_low, 'ro', label=f"Undersampled ({fs_low} Hz) - Aliasing", zorder=3)

    # Resampled signal
    plt.plot(t_resample, signal_resample, 'mo', label=f"Resampled ({fs_resample} Hz)", zorder=3)

    plt.xlabel("Time (s)")
    plt.ylabel("Amplitude")
    plt.title("Effect of Sampling Below the Nyquist Frequency")
    plt.legend()
    plt.grid(True)
    plt.show()

    # Plot the interpolated signals
    plt.figure(figsize=(12, 6))

    # Original continuous signal
    plt.plot(t_continuous, signal_continuous, 'k', alpha=0.5, label="Original Signal (1000 Hz)")

    # Interpolated signals
    plt.plot(t_continuous, signal_high_interp, 'b-', label=f"Interpolated from {fs_high} Hz", zorder=3)
    plt.plot(t_continuous, signal_nyquist_interp, 'g-', label=f"Interpolated from {fs_nyquist} Hz", zorder=3)
    plt.plot(t_continuous, signal_low_interp, 'r-', label=f"Interpolated from {fs_low} Hz", zorder=3)
    plt.plot(t_continuous, signal_resample_interp, 'm-', label=f"Interpolated from {fs_resample} Hz", zorder=3)

    plt.xlabel("Time (s)")
    plt.ylabel("Amplitude")
    plt.title("Sinc Interpolation of Sampled Signals")
    plt.legend()
    plt.grid(True)
    plt.show()

    # Compute and plot power spectrum
    freqs_continuous, power_continuous = compute_power_spectrum(signal_continuous, fs_continuous)
    freqs_low_interp, power_low_interp = compute_power_spectrum(signal_low_interp, fs_continuous)
    freqs_resample_interp, power_resample_interp = compute_power_spectrum(signal_resample_interp, fs_continuous)

    # Normalize power spectra
    max_power_continuous = np.max(power_continuous)
    power_continuous -= max_power_continuous
    power_low_interp -= max_power_continuous
    power_resample_interp -= max_power_continuous

    plt.figure(figsize=(12, 6))
    plt.plot(freqs_continuous, power_continuous, 'k', label="Original Signal (1000 Hz)")
    plt.plot(freqs_low_interp, power_low_interp, 'r', label=f"Interpolated from {fs_low} Hz")
    plt.plot(freqs_resample_interp, power_resample_interp, 'm', label=f"Interpolated from {fs_resample} Hz")

    plt.xlabel("Frequency (Hz)")
    plt.ylabel("Power (dB)")
    plt.title("Power Spectrum of Signals")
    plt.legend()
    plt.grid(True)
    plt.show()

# Interactive widget
interact(update_plot, fs_resample=IntSlider(min=5, max=100, step=5, value=50));

interactive(children=(IntSlider(value=50, description='fs_resample', min=5, step=5), Output()), _dom_classes=(…